# WAV_car_MySQL_적재

## 0. Spark Session 생성

In [2]:
from pyspark.sql import SparkSession
import config
from config import DB_CONFIG, HDFS_CONFIG

# MySQL JDBC 드라이버 경로
mysql_driver_path = config.MYSQL_JDBC

# SparkSession 생성
spark = SparkSession.builder \
    .appName("WAV_sql_load") \
    .config("spark.hadoop.fs.defaultFS", HDFS_CONFIG["defaultFS"]) \
    .config("spark.driver.memory", "8g") \
    .config("spark.executor.memory", "8g") \
    .config("spark.driver.maxResultSize", "4g") \
    .config("spark.jars", mysql_driver_path) \
    .config("spark.sql.execution.arrow.pyspark.enabled", "true")\
    .getOrCreate()

In [2]:
import sys
print("Python version:", sys.version)

Python version: 3.6.13 |Anaconda, Inc.| (default, Jun  4 2021, 14:25:59) 
[GCC 7.5.0]


## 1. wav_car_horn_data

### 1.1 WAV -> MFCC 변환 데이터프레임

In [2]:
import pyspark
from pyspark.sql import SparkSession
import io
from scipy.io import wavfile
import librosa
import numpy as np
import os
from pyspark.sql.functions import udf
from pyspark.sql.types import ArrayType, FloatType, StringType

# HDFS에서 모든 WAV 파일 읽기
hdfs_dir = f"{config.HDFS_BASE_PATH}/raw_data/1.Car/1.horn_of_car"
binary_df = spark.read.format("binaryFile").load(hdfs_dir)

# 🔹 UDF (User Defined Function) 정의: WAV → MFCC 변환
def extract_mfcc(binary_data):
    try:
        audio_bytes = io.BytesIO(binary_data)  # 바이너리 데이터를 메모리 파일로 변환
        sr, audio = wavfile.read(audio_bytes)  # scipy로 샘플링 레이트 확인
        audio_librosa, sr_librosa = librosa.load(audio_bytes, sr=None)  # librosa로 리샘플링
        mfcc = librosa.feature.mfcc(y=audio_librosa, sr=sr_librosa, n_mfcc=50)  # MFCC 추출
        mfcc_mean = np.mean(mfcc, axis=1).astype(float)  # 평균 계산
        return mfcc_mean.tolist()  # 리스트로 반환
    except Exception as e:
        return None  # 에러 발생 시 None 반환

# UDF 등록
mfcc_udf = udf(extract_mfcc, ArrayType(FloatType()))

# 🔹 파일 이름 추출 UDF 정의
def extract_filename(path):
    return os.path.basename(path)

filename_udf = udf(extract_filename, StringType())

# 🔹 변환 적용
df_mfcc = binary_df \
    .withColumn("fileName", filename_udf(binary_df["path"])) \
    .withColumn("mfcc_features", mfcc_udf(binary_df["content"]))

# 🔹 배열 데이터를 개별 컬럼으로 변환
mfcc_columns = [f"mfcc_{i+1}" for i in range(50)]
for i in range(50):
    df_mfcc = df_mfcc.withColumn(mfcc_columns[i], df_mfcc["mfcc_features"][i])

# 🔹 불필요한 컬럼 정리
df_mfcc = df_mfcc.select(["fileName"] + mfcc_columns)

# 🔹 결과 저장 (HDFS)
# output_path = "hdfs://localhost:9000/shared_data/mfcc_features/"
# df_mfcc.write.csv(output_path, header=True, mode="overwrite")

# print(f"✅ MFCC 데이터가 HDFS에 저장됨: {output_path}")

In [3]:
df_mfcc.printSchema()

root
 |-- fileName: string (nullable = true)
 |-- mfcc_1: float (nullable = true)
 |-- mfcc_2: float (nullable = true)
 |-- mfcc_3: float (nullable = true)
 |-- mfcc_4: float (nullable = true)
 |-- mfcc_5: float (nullable = true)
 |-- mfcc_6: float (nullable = true)
 |-- mfcc_7: float (nullable = true)
 |-- mfcc_8: float (nullable = true)
 |-- mfcc_9: float (nullable = true)
 |-- mfcc_10: float (nullable = true)
 |-- mfcc_11: float (nullable = true)
 |-- mfcc_12: float (nullable = true)
 |-- mfcc_13: float (nullable = true)
 |-- mfcc_14: float (nullable = true)
 |-- mfcc_15: float (nullable = true)
 |-- mfcc_16: float (nullable = true)
 |-- mfcc_17: float (nullable = true)
 |-- mfcc_18: float (nullable = true)
 |-- mfcc_19: float (nullable = true)
 |-- mfcc_20: float (nullable = true)
 |-- mfcc_21: float (nullable = true)
 |-- mfcc_22: float (nullable = true)
 |-- mfcc_23: float (nullable = true)
 |-- mfcc_24: float (nullable = true)
 |-- mfcc_25: float (nullable = true)
 |-- mfcc_26: 

In [5]:
df_mfcc.count()

3189

### 1.2 wav_car_horn_data 데이터 MySQL에 적재

In [6]:
df_mfcc.createOrReplaceTempView("wav_car_horn50")

In [7]:
# SQL 쿼리 실행하여 데이터 추출
df_mfcc = spark.sql("""
    SELECT *
    FROM wav_car_horn50
""")

In [8]:
# ✅ Config 파일에서 MySQL 연결 정보 로드
mysql_url = f"jdbc:mysql://{DB_CONFIG['host']}:{DB_CONFIG['port']}/{DB_CONFIG['database']}?useUnicode=true&characterEncoding=UTF-8"
mysql_properties = {
    "user": DB_CONFIG["user"],
    "password": DB_CONFIG["password"],
    "driver": "com.mysql.cj.jdbc.Driver"
}

In [9]:
# MySQL로 DataFrame 적재 (쿼리 결과가 None이 아닌 경우에만)
if df_mfcc is not None:
    df_mfcc.write.jdbc(url=mysql_url, table="wav_car_horn_data50", mode="overwrite", properties=mysql_properties)
    print("데이터가 MySQL로 성공적으로 적재되었습니다!")
else:
    print("쿼리 결과가 없습니다. 데이터 추출이 실패했습니다.")


데이터가 MySQL로 성공적으로 적재되었습니다!


## 2. wav_car_siren_data

### 2.1 WAV -> MFCC 변환 데이터프레임

In [10]:
import pyspark
from pyspark.sql import SparkSession
import io
from scipy.io import wavfile
import librosa
import numpy as np
import os
from pyspark.sql.functions import udf
from pyspark.sql.types import ArrayType, FloatType, StringType

# HDFS에서 모든 WAV 파일 읽기
hdfs_dir = f"{config.HDFS_BASE_PATH}/raw_data/1.Car/2.siren_of_car"
binary_df = spark.read.format("binaryFile").load(hdfs_dir)

# 🔹 UDF (User Defined Function) 정의: WAV → MFCC 변환
def extract_mfcc(binary_data):
    try:
        audio_bytes = io.BytesIO(binary_data)  # 바이너리 데이터를 메모리 파일로 변환
        sr, audio = wavfile.read(audio_bytes)  # scipy로 샘플링 레이트 확인
        audio_librosa, sr_librosa = librosa.load(audio_bytes, sr=None)  # librosa로 리샘플링
        mfcc = librosa.feature.mfcc(y=audio_librosa, sr=sr_librosa, n_mfcc=50)  # MFCC 추출
        mfcc_mean = np.mean(mfcc, axis=1).astype(float)  # 평균 계산
        return mfcc_mean.tolist()  # 리스트로 반환
    except Exception as e:
        return None  # 에러 발생 시 None 반환

# UDF 등록
mfcc_udf = udf(extract_mfcc, ArrayType(FloatType()))

# 🔹 파일 이름 추출 UDF 정의
def extract_filename(path):
    return os.path.basename(path)

filename_udf = udf(extract_filename, StringType())

# 🔹 변환 적용
df_mfcc_siren_car = binary_df \
    .withColumn("fileName", filename_udf(binary_df["path"])) \
    .withColumn("mfcc_features", mfcc_udf(binary_df["content"]))

# 🔹 배열 데이터를 개별 컬럼으로 변환
mfcc_columns = [f"mfcc_{i+1}" for i in range(50)]
for i in range(50):
    df_mfcc_siren_car = df_mfcc_siren_car.withColumn(mfcc_columns[i], df_mfcc_siren_car["mfcc_features"][i])

# 🔹 불필요한 컬럼 정리
df_mfcc_siren_car = df_mfcc_siren_car.select(["fileName"] + mfcc_columns)

# 🔹 결과 저장 (HDFS)
# output_path = "hdfs://localhost:9000/shared_data/mfcc_features/"
# df_mfcc.write.csv(output_path, header=True, mode="overwrite")

# print(f"✅ MFCC 데이터가 HDFS에 저장됨: {output_path}")

In [12]:
df_mfcc_siren_car.printSchema()

root
 |-- fileName: string (nullable = true)
 |-- mfcc_1: float (nullable = true)
 |-- mfcc_2: float (nullable = true)
 |-- mfcc_3: float (nullable = true)
 |-- mfcc_4: float (nullable = true)
 |-- mfcc_5: float (nullable = true)
 |-- mfcc_6: float (nullable = true)
 |-- mfcc_7: float (nullable = true)
 |-- mfcc_8: float (nullable = true)
 |-- mfcc_9: float (nullable = true)
 |-- mfcc_10: float (nullable = true)
 |-- mfcc_11: float (nullable = true)
 |-- mfcc_12: float (nullable = true)
 |-- mfcc_13: float (nullable = true)
 |-- mfcc_14: float (nullable = true)
 |-- mfcc_15: float (nullable = true)
 |-- mfcc_16: float (nullable = true)
 |-- mfcc_17: float (nullable = true)
 |-- mfcc_18: float (nullable = true)
 |-- mfcc_19: float (nullable = true)
 |-- mfcc_20: float (nullable = true)
 |-- mfcc_21: float (nullable = true)
 |-- mfcc_22: float (nullable = true)
 |-- mfcc_23: float (nullable = true)
 |-- mfcc_24: float (nullable = true)
 |-- mfcc_25: float (nullable = true)
 |-- mfcc_26: 

In [13]:
df_mfcc_siren_car.count()

1990

### 2.2 wav_car_siren_data 데이터 MySQL에 적재

In [14]:
df_mfcc_siren_car.createOrReplaceTempView("wav_car_siren50")

# SQL 쿼리 실행하여 데이터 추출
df_mfcc_siren_car = spark.sql("""
    SELECT *
    FROM wav_car_siren50
""")

# ✅ Config 파일에서 MySQL 연결 정보 로드
mysql_url = f"jdbc:mysql://{DB_CONFIG['host']}:{DB_CONFIG['port']}/{DB_CONFIG['database']}?useUnicode=true&characterEncoding=UTF-8"
mysql_properties = {
    "user": DB_CONFIG["user"],
    "password": DB_CONFIG["password"],
    "driver": "com.mysql.cj.jdbc.Driver"
}

# MySQL로 DataFrame 적재 (쿼리 결과가 None이 아닌 경우에만)
if df_mfcc_siren_car is not None:
    df_mfcc_siren_car.write.jdbc(url=mysql_url, table="wav_car_siren_data50", mode="overwrite", properties=mysql_properties)
    print("데이터가 MySQL로 성공적으로 적재되었습니다!")
else:
    print("쿼리 결과가 없습니다. 데이터 추출이 실패했습니다.")


데이터가 MySQL로 성공적으로 적재되었습니다!


## 3. wav_car_driving_data

### 3.1 WAV -> MFCC 변환 데이터프레임

In [2]:
import pyspark
from pyspark.sql import SparkSession
import io
from scipy.io import wavfile
import librosa
import numpy as np
import os
from pyspark.sql.functions import udf
from pyspark.sql.types import ArrayType, FloatType, StringType

# HDFS에서 모든 WAV 파일 읽기
hdfs_dir = f"{config.HDFS_BASE_PATH}/raw_data/1.Car/3.driving_sound_of_car"
binary_df = spark.read.format("binaryFile").load(hdfs_dir)

# 🔹 UDF (User Defined Function) 정의: WAV → MFCC 변환
def extract_mfcc(binary_data):
    try:
        audio_bytes = io.BytesIO(binary_data)  # 바이너리 데이터를 메모리 파일로 변환
        sr, audio = wavfile.read(audio_bytes)  # scipy로 샘플링 레이트 확인
        audio_librosa, sr_librosa = librosa.load(audio_bytes, sr=None)  # librosa로 리샘플링
        mfcc = librosa.feature.mfcc(y=audio_librosa, sr=sr_librosa, n_mfcc=50)  # MFCC 추출
        mfcc_mean = np.mean(mfcc, axis=1).astype(float)  # 평균 계산
        return mfcc_mean.tolist()  # 리스트로 반환
    except Exception as e:
        return None  # 에러 발생 시 None 반환

# UDF 등록
mfcc_udf = udf(extract_mfcc, ArrayType(FloatType()))

# 🔹 파일 이름 추출 UDF 정의
def extract_filename(path):
    return os.path.basename(path)

filename_udf = udf(extract_filename, StringType())

# 🔹 변환 적용
df_mfcc_driving_car = binary_df \
    .withColumn("fileName", filename_udf(binary_df["path"])) \
    .withColumn("mfcc_features", mfcc_udf(binary_df["content"]))

# 🔹 배열 데이터를 개별 컬럼으로 변환
mfcc_columns = [f"mfcc_{i+1}" for i in range(50)]
for i in range(50):
    df_mfcc_driving_car = df_mfcc_driving_car.withColumn(mfcc_columns[i], df_mfcc_driving_car["mfcc_features"][i])

# 🔹 불필요한 컬럼 정리
df_mfcc_driving_car = df_mfcc_driving_car.select(["fileName"] + mfcc_columns)

# 🔹 결과 저장 (HDFS)
# output_path = "hdfs://localhost:9000/shared_data/mfcc_features/"
# df_mfcc.write.csv(output_path, header=True, mode="overwrite")

# print(f"✅ MFCC 데이터가 HDFS에 저장됨: {output_path}")

In [3]:
df_mfcc_driving_car.printSchema()

root
 |-- fileName: string (nullable = true)
 |-- mfcc_1: float (nullable = true)
 |-- mfcc_2: float (nullable = true)
 |-- mfcc_3: float (nullable = true)
 |-- mfcc_4: float (nullable = true)
 |-- mfcc_5: float (nullable = true)
 |-- mfcc_6: float (nullable = true)
 |-- mfcc_7: float (nullable = true)
 |-- mfcc_8: float (nullable = true)
 |-- mfcc_9: float (nullable = true)
 |-- mfcc_10: float (nullable = true)
 |-- mfcc_11: float (nullable = true)
 |-- mfcc_12: float (nullable = true)
 |-- mfcc_13: float (nullable = true)
 |-- mfcc_14: float (nullable = true)
 |-- mfcc_15: float (nullable = true)
 |-- mfcc_16: float (nullable = true)
 |-- mfcc_17: float (nullable = true)
 |-- mfcc_18: float (nullable = true)
 |-- mfcc_19: float (nullable = true)
 |-- mfcc_20: float (nullable = true)
 |-- mfcc_21: float (nullable = true)
 |-- mfcc_22: float (nullable = true)
 |-- mfcc_23: float (nullable = true)
 |-- mfcc_24: float (nullable = true)
 |-- mfcc_25: float (nullable = true)
 |-- mfcc_26: 

In [4]:
df_mfcc_driving_car.count()

1682

### 3.2 wav_car_siren_data 데이터 MySQL에 적재

In [5]:
df_mfcc_driving_car.createOrReplaceTempView("wav_car_driving50")

# SQL 쿼리 실행하여 데이터 추출
df_mfcc_driving_car = spark.sql("""
    SELECT *
    FROM wav_car_driving50
""")

# ✅ Config 파일에서 MySQL 연결 정보 로드
mysql_url = f"jdbc:mysql://{DB_CONFIG['host']}:{DB_CONFIG['port']}/{DB_CONFIG['database']}?useUnicode=true&characterEncoding=UTF-8"
mysql_properties = {
    "user": DB_CONFIG["user"],
    "password": DB_CONFIG["password"],
    "driver": "com.mysql.cj.jdbc.Driver"
}

# MySQL로 DataFrame 적재 (쿼리 결과가 None이 아닌 경우에만)
if df_mfcc_driving_car is not None:
    df_mfcc_driving_car.write.jdbc(url=mysql_url, table="wav_car_driving_data50", mode="overwrite", properties=mysql_properties)
    print("데이터가 MySQL로 성공적으로 적재되었습니다!")
else:
    print("쿼리 결과가 없습니다. 데이터 추출이 실패했습니다.")


데이터가 MySQL로 성공적으로 적재되었습니다!


In [3]:
spark.stop()